In [1]:
import os

from fptools.io import load_data
from fptools.preprocess.pipelines import lowpass_dff
from fptools.io.test import download_test_data, list_datasets

## Let's download some test/demo data
The following code will download a demo dataset to your local computer, if is does not already exist. The variable `test_data_path` will contain the path to the root of the test dataset folder. If the parameter `dest` is `None`, we will pick a location within the current user's home directory.

Use the function `list_datasets()` to list the names of the available datasets within the demo data.

In [2]:
test_data_path = download_test_data(dest=None)
list_datasets()

Test data appears to already be in place at "C:\Users\thackray\fptools_test_data".


['MA-PR4-4Day', 'TDT-DLS-GRABDA2m-Male-PR4-2Day']

## Time to load the data

The `load_data()` function from `fptools.io` is your main gateway to loading data in a variety of formats. Shown below is an exmaple of loading data from a TDT tank.

The `load_data()` function will return you a `SessionCollection`, which is really a `list` but with extra functionality built-in. This means you can largely use it any way you would use a normal python list. 

In [3]:
# path to the dataset we want to load
tank_path = os.path.join(test_data_path, "TDT-DLS-GRABDA2m-Male-PR4-2Day")

signal_map = [{
    'tdt_name': '_465A',
    'dest_name': 'Dopamine',
    'role': 'experimental'
}, {
    'tdt_name': '_415A',
    'dest_name': 'Isosbestic',
    'role': 'control'
}]

sessions = load_data(tank_path,
                     signal_map,

                     # path to a manifest file for additional metadata to inject into sessions
                     # manifest index tells the system how to key the manifest and assoicate to data
                     # for TDT data, Sessions are named as the blockname.
                     manifest_path=os.path.join(tank_path, 'manifest.xlsx'),
                     manifest_index='blockname',

                     # use multiple worker processes to load data in parallel
                     max_workers=4,

                     # The locator tells the system what type of data we are looking for.
                     # more complex functionality is available, but most folks could use the string "tdt" for TDT tank files
                     # or the string "ma" for med-associates style files.
                     locator="tdt",

                     # a function to run any preprocessing needed (see library of pre-made pipelines in `fptools.preprocess.pipelines`)
                     preprocess=lowpass_dff,

                     # control if we want to cache files, and where that cache should live
                     cache=True,
                     cache_dir=os.path.join(tank_path, 'cache_lowpass_dff'),

                     # these options will be passed along to the preprocess routine
                     show_steps=True,
                     plot_dir=os.path.join(tank_path, 'cache_lowpass_dff'),
                     downsample=10)

sessions.rename_epoc('RNP_', 'RNP')
sessions.rename_epoc('RMG_', 'RMG')

  0%|          | 0/16 [00:00<?, ?it/s]

## Get an overview of `SessionCollection` contents

The `SessionCollection.describe()` method will print you a string summary of the datasets contained within the collection. Next to each dataset, the number of sessions with having that dataset is provided within the parenthesis.

In [4]:
sessions.describe()

Number of sessions: 16

Signals present in data with counts:
(16) "Dopamine"
(16) "Fi1d"
(16) "Fi1r"
(16) "Isosbestic"
(16) "_415A"
(16) "_465A"

Epocs present in data with counts:
(16) "Cam1"
(16) "Nose"
(16) "P1SC"
(16) "Tick"
(16) "URM_"
(16) "UnNP"
(16) "RNP"
(16) "RMG"

Scalars present in data with counts:
(16) "Fi1i"




## Get an overview of `Session` contents.

Similar to `SessionCollection.describe()`, there is also a `Session.describe()`. The information here is a bit more detailed than at the collection level. Where possibe, we provide summary statistics for each dataset.

In [5]:
sessions[0].describe()

Session with name "EN140_PRD2-240814-140637"

Metadata:
    blockname: EN140_PRD2-240814-140637
    duration: 0:59:59.194253
    snip_channel: 0
    start_date: 2024-08-14 14:06:42.999999
    stop_date: 2024-08-14 15:06:42.194252
    stream_channel: 0
    tankpath: C:\Users\thackray\fptools_test_data\TDT-DLS-GRABDA2m-Male-PR4-2Day
    utc_start_time: 14:06:42
    utc_stop_time: 15:06:42
    mouseID: EN140
    sex: M
    genotype: HET
    sensor: GRABDA2M
    hemisphere: L
    paradigm: PR4
    paradigm_day: 2
    cable in?: True
    cube: TDT1
    exclude: False
    notes: nan

Epocs:
    Cam1:
        num_events = (11999,)
        avg_rate = 0:00:00.048824
        earliest = 0:00:00.086016
        latest = 0:09:59.984701
    Nose:
        num_events = (242,)
        avg_rate = 0:00:00.209715
        earliest = 0:00:00.644383
        latest = 0:09:28.401265
    P1SC:
        num_events = (23,)
        avg_rate = 0:00:03.015475
        earliest = 0:00:00.666829
        latest = 0:09:30.

## Fetch metadata across sessions as a DataFrame

Use the `SessionCollection.metadata` property to retrieve a `pandas.DataFrame` containing metadata across sessions. Note: changing values in this dataframe will not have any effect on the underlying data. Instead, make these changes on the underlying `Session.metadata` property.

In [6]:
sessions.metadata

,blockname,duration,snip_channel,start_date,stop_date,stream_channel,tankpath,utc_start_time,utc_stop_time,mouseID,sex,genotype,sensor,hemisphere,paradigm,paradigm_day,cable in?,cube,exclude,notes
0,EN140_PRD2-240814-140637,0 days 00:59:59.194253,0,2024-08-14 14:06:42.999999,2024-08-14 15:06:42.194252,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,14:06:42,15:06:42,EN140,M,HET,GRABDA2M,L,PR4,2,True,TDT1,False,NaN
1,EN140_PRD1-240813-133324,0 days 00:59:59.101191,0,2024-08-13 13:33:28.999999,2024-08-13 14:33:28.101190,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,13:33:28,14:33:28,EN140,M,HET,GRABDA2M,L,PR4,1,True,TDT1,False,NaN
2,EN157_PRD1-240814-125543,0 days 00:59:59.199823,0,2024-08-14 12:55:48.999999,2024-08-14 13:55:48.199822,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,12:55:48,13:55:48,EN157,M,WTY,GRABDA2M,R,PR4,1,True,TDT1,False,NaN
3,EN157_PRD2-240815-122504,0 days 00:59:59.098734,0,2024-08-15 12:25:08.999999,2024-08-15 13:25:08.098733,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,12:25:08,13:25:08,EN157,M,WTY,GRABDA2M,R,PR4,2,True,TDT1,False,NaN
4,EN165_PRD1-240818-150154,0 days 00:59:59.078418,0,2024-08-18 15:01:59.999999,2024-08-18 16:01:59.078417,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,15:01:59,16:01:59,EN165,M,WTY,GRABDA2M,R,PR4,1,True,TDT2,False,NaN
5,EN165_PRD2-240819-144305,0 days 00:59:59.081367,0,2024-08-19 14:43:10.999999,2024-08-19 15:43:10.081366,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,14:43:10,15:43:10,EN165,M,WTY,GRABDA2M,R,PR4,2,True,TDT2,False,NaN
6,EN171_PRD1-240815-144154,0 days 00:59:59.085299,0,2024-08-15 14:41:59.999999,2024-08-15 15:41:59.085298,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,14:41:59,15:41:59,EN171,M,HET,GRABDA2M,L,PR4,1,True,TDT2,False,NaN
7,EN171_PRD2-240816-125119,0 days 00:59:59.078090,0,2024-08-16 12:51:24.999999,2024-08-16 13:51:24.078089,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,12:51:24,13:51:24,EN171,M,HET,GRABDA2M,L,PR4,2,True,TDT2,False,NaN
8,EN172_PRD1-240814-104140,0 days 00:59:59.094146,0,2024-08-14 10:41:45.999999,2024-08-14 11:41:45.094145,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,10:41:45,11:41:45,EN172,M,WTY,GRABDA2M,R,PR4,1,True,TDT2,False,NaN
9,EN172_PRD2-240815-111039,0 days 00:59:59.231936,0,2024-08-15 11:10:44.999999,2024-08-15 12:10:44.231935,0,C:\Users\thackray\fptools_test_data\TDT-DLS-GR...,11:10:44,12:10:44,EN172,M,WTY,GRABDA2M,R,PR4,2,True,TDT2,False,NaN
